In [1]:
import requests
import pandas as pd
from datetime import datetime
import re 

In [2]:
# Brooklyn: (40.6571, -73.9444), Bronx: (40.831, -73.9109), Queens: (40.757, -73.8843) 
# Manhattan: (40.7777, -73.966), Staten Island: (40.5981, -74.1344) 

geo_list = [(40.6571, -73.9444), (40.831, -73.9109), (40.757, -73.8843), (40.7777, -73.966), 
            (40.5981, -74.1344)] 

In [35]:
# Function to fetch forecast for a single lat/long point
def fetch_forecast(latitude, longitude):
    # Get grid points for the specified lat/long from the NWS API
    grid_points_url = f"https://api.weather.gov/points/{latitude},{longitude}"
    response = requests.get(grid_points_url)
    data = response.json()

    # Extract the gridId and gridX/gridY
    grid_id = data['properties']['gridId']
    grid_x = data['properties']['gridX']
    grid_y = data['properties']['gridY']

    # Fetch the forecast using gridId, gridX, and gridY
    forecast_url = f"https://api.weather.gov/gridpoints/{grid_id}/{grid_x},{grid_y}/forecast"
    forecast_response = requests.get(forecast_url)
    forecast_data = forecast_response.json()

    # Return forecast periods
    return forecast_data['properties']['periods']

# Function to convert text precipitation forecast to inches
def convert_to_inches(value):
    value = value.lower()
    if value in ['a quarter', 'quarter']:
        return 0.25
    elif value == 'half':
        return 0.5
    elif value in ['a tenth', 'tenth']:
        return 0.1
    else:
        try:
            return float(value)
        except ValueError:
            return 0.0

# Function to extract precipitation amounts (rain/snow) and wind speed from forecast text
def extract_precipitation_amount(forecast_text, type_='rain'):
    try:
        if type_ == 'rain':
            # Match phrases like "between a tenth and quarter of an inch"
            range_match = re.search(r"between\s+(a\s+tenth|tenth|a\s+quarter|quarter|[\d\.]+)\s+and\s+(a\s+quarter|quarter|half|tenth|a\s+tenth|[\d\.]+)\s+of\s+an\s+inch", forecast_text, re.IGNORECASE)
            single_match = re.search(r"(\d*\.?\d+|a\s+quarter|quarter|a\s+tenth|tenth|half)\s*(inches|inch)", forecast_text, re.IGNORECASE)

            if range_match:
                # Convert words like "a quarter," "quarter," "half," or "a tenth" to numeric values
                start = convert_to_inches(range_match.group(1))
                end = convert_to_inches(range_match.group(2))
                return (start + end) / 2  # Return the average of the range
            elif single_match:
                return convert_to_inches(single_match.group(1))
        elif type_ == 'snow':
            # Match phrases like "2 inches snow", "between 1 and 3 inches of snow"
            match = re.search(r"(\d*\.?\d+|a\s+quarter|quarter|a\s+tenth|tenth|half)\s*(inches|inch)(\s+of\s+)?\s*snow", forecast_text, re.IGNORECASE)
            return convert_to_inches(match.group(1)) if match else 0.0
        else:
            raise ValueError("Unsupported precipitation type. Use 'rain' or 'snow'.")
    except Exception as e:
        print(f"Error extracting precipitation amount: {e}")
        return 0.0

# Function to extract wind speed from forecast text
def extract_wind_speed(forecast_text):
    match = re.search(r"(\d+)\s*mph", forecast_text, re.IGNORECASE)
    return int(match.group(1)) if match else 0  # Return 0 if no wind speed is found

# Function to get min/max temp, wind, rainfall, and snow amount per day
def get_daily_forecast(forecast_periods):
    daily_data = []
    added_dates = []  # Keep track of dates we've added to avoid duplicates
    
    for period in forecast_periods:
        # Extract date from 'startTime'
        start_time_str = period['startTime']
        start_time = datetime.strptime(start_time_str, '%Y-%m-%dT%H:%M:%S%z')
        date = start_time.date()

        # We only want one value per day, skip if we've already added that date
        if date in added_dates:
            continue
        
        # Extract temperature
        temperature = period['temperature']
        temperature_unit = period['temperatureUnit']
        #humidity = period['relativeHumidity']
        
        # Extract rainfall and snow descriptions and amounts
        rainfall_info = period['detailedForecast']
        
        rainfall_amount = extract_precipitation_amount(rainfall_info, type_='rain')
        snow_amount = extract_precipitation_amount(rainfall_info, type_='snow')

        # Extract wind description and speed
        wind_speed = extract_wind_speed(rainfall_info)
        
        # Collect the min/max temperature for the day
        daily_min_temp = period['temperature']  # Initial assumption
        daily_max_temp = period['temperature']  # Initial assumption
        
        # Iterate over periods of the same day to find min and max temps
        for next_period in forecast_periods:
            next_period_date = datetime.strptime(next_period['startTime'], '%Y-%m-%dT%H:%M:%S%z').date()
            if next_period_date == date:
                # Update min/max temperature
                daily_min_temp = min(daily_min_temp, next_period['temperature'])
                daily_max_temp = max(daily_max_temp, next_period['temperature'])
        
        # Append data for this day
        daily_data.append({
            'date': date.strftime('%m/%d/%Y'),
            'min_temp': daily_min_temp,
            'max_temp': daily_max_temp,
            'temperature_unit': temperature_unit,
            #'humidity': humidity,
            'rainfall_inches': rainfall_amount,
            'snow_inches': snow_amount,
            'wind_speed_mph': wind_speed,
            'text': rainfall_info
        })
                
        # Mark this date as added
        added_dates.append(date)
        
        # Stop when we have 3 days of data
        if len(daily_data) == 3:
            break

    return daily_data



In [36]:
# Initialize an empty list to store forecast data for all points
all_forecasts = []

# Iterate through geo_list and fetch forecast for each point
for latitude, longitude in geo_list:
    print(f"Fetching forecast for location: ({latitude}, {longitude})")
    forecast_periods = fetch_forecast(latitude, longitude)
    
    if forecast_periods:
        # Get 3-day outlook for this point (one value per day)
        daily_outlook = get_daily_forecast(forecast_periods)
        print(daily_outlook)
        
        # Add location info to each day's forecast
        for day_forecast in daily_outlook:
            day_forecast['latitude'] = latitude
            day_forecast['longitude'] = longitude
             
        
        # Append the 3-day outlook for this location to all_forecasts
        all_forecasts.extend(daily_outlook)
    else:
        print(f"Failed to get forecast for ({latitude}, {longitude})\n")


all_forecasts

Fetching forecast for location: (40.6571, -73.9444)
[{'date': '12/15/2024', 'min_temp': 37, 'max_temp': 37, 'temperature_unit': 'F', 'rainfall_inches': 0.375, 'snow_inches': 0.0, 'wind_speed_mph': 7, 'text': 'Rain after 10pm. Cloudy. Low around 37, with temperatures rising to around 41 overnight. East wind around 7 mph. Chance of precipitation is 100%. New rainfall amounts between a quarter and half of an inch possible.'}, {'date': '12/16/2024', 'min_temp': 50, 'max_temp': 51, 'temperature_unit': 'F', 'rainfall_inches': 0.375, 'snow_inches': 0.0, 'wind_speed_mph': 10, 'text': 'Rain. Cloudy. High near 51, with temperatures falling to around 49 in the afternoon. Southeast wind 6 to 10 mph. Chance of precipitation is 100%. New rainfall amounts between a quarter and half of an inch possible.'}, {'date': '12/17/2024', 'min_temp': 40, 'max_temp': 59, 'temperature_unit': 'F', 'rainfall_inches': 0.175, 'snow_inches': 0.0, 'wind_speed_mph': 13, 'text': 'A chance of rain before 10am. Partly sunn

[{'date': '12/15/2024',
  'min_temp': 37,
  'max_temp': 37,
  'temperature_unit': 'F',
  'rainfall_inches': 0.375,
  'snow_inches': 0.0,
  'wind_speed_mph': 7,
  'text': 'Rain after 10pm. Cloudy. Low around 37, with temperatures rising to around 41 overnight. East wind around 7 mph. Chance of precipitation is 100%. New rainfall amounts between a quarter and half of an inch possible.',
  'latitude': 40.6571,
  'longitude': -73.9444},
 {'date': '12/16/2024',
  'min_temp': 50,
  'max_temp': 51,
  'temperature_unit': 'F',
  'rainfall_inches': 0.375,
  'snow_inches': 0.0,
  'wind_speed_mph': 10,
  'text': 'Rain. Cloudy. High near 51, with temperatures falling to around 49 in the afternoon. Southeast wind 6 to 10 mph. Chance of precipitation is 100%. New rainfall amounts between a quarter and half of an inch possible.',
  'latitude': 40.6571,
  'longitude': -73.9444},
 {'date': '12/17/2024',
  'min_temp': 40,
  'max_temp': 59,
  'temperature_unit': 'F',
  'rainfall_inches': 0.175,
  'snow_in

In [37]:
# Create a pandas DataFrame from the collected forecast data
df = pd.DataFrame(all_forecasts)

# Dictionary mapping coordinates to boroughs
boroughs = {
    (40.6571, -73.9444): 'Brooklyn',
    (40.831, -73.9109): 'Bronx',
    (40.757, -73.8843): 'Queens',
    (40.7777, -73.966): 'Manhattan',
    (40.5981, -74.1344): 'Staten Island'
}

# Function to map latitude and longitude to borough
def get_borough(row):
    return boroughs.get((row['latitude'], row['longitude']), 'Unknown')

# Apply the function to add the 'borough' column
df['borough'] = df.apply(get_borough, axis=1)

df

,date,min_temp,max_temp,temperature_unit,rainfall_inches,snow_inches,wind_speed_mph,text,latitude,longitude,borough
0,12/15/2024,37,37,F,0.375,0.0,7,"Rain after 10pm. Cloudy. Low around 37, with t...",40.6571,-73.9444,Brooklyn
1,12/16/2024,50,51,F,0.375,0.0,10,"Rain. Cloudy. High near 51, with temperatures ...",40.6571,-73.9444,Brooklyn
2,12/17/2024,40,59,F,0.175,0.0,13,A chance of rain before 10am. Partly sunny. Hi...,40.6571,-73.9444,Brooklyn
3,12/15/2024,37,37,F,0.175,0.0,7,"Rain after 10pm. Cloudy. Low around 37, with t...",40.8310,-73.9109,Bronx
4,12/16/2024,49,50,F,0.375,0.0,9,"Rain. Cloudy. High near 50, with temperatures ...",40.8310,-73.9109,Bronx
5,12/17/2024,40,59,F,0.175,0.0,13,A chance of rain before 10am. Partly sunny. Hi...,40.8310,-73.9109,Bronx
6,12/15/2024,37,37,F,0.175,0.0,7,"Rain after 10pm. Cloudy. Low around 37, with t...",40.7570,-73.8843,Queens
7,12/16/2024,49,50,F,0.375,0.0,10,"Rain. Cloudy. High near 50, with temperatures ...",40.7570,-73.8843,Queens
8,12/17/2024,41,60,F,0.175,0.0,14,A chance of rain before 10am. Partly sunny. Hi...,40.7570,-73.8843,Queens
9,12/15/2024,37,37,F,0.375,0.0,5,"Rain after 10pm. Cloudy. Low around 37, with t...",40.7777,-73.9660,Manhattan
